## Домашнее задание 2

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
Exposure             115155 non-null float64
LicAge               115155 non-null int64
RecordBeg            115155 non-null object
RecordEnd            59455 non-null object
Gender               115155 non-null object
MariStat             115155 non-null object
SocioCateg           115155 non-null object
VehUsage             115155 non-null object
DrivAge              115155 non-null int64
HasKmLimit           115155 non-null int64
BonusMalus           115155 non-null int64
ClaimAmount          115155 non-null float64
ClaimInd             115155 non-null int64
ClaimNbResp          115155 non-null float64
ClaimNbNonResp       115155 non-null float64
ClaimNbParking       115155 non-null float64
ClaimNbFireTheft     115155 non-null float64
ClaimNbWindscreen    115155 non-null float64
OutUseNb             115155 non-null float64
RiskArea             115155 non-null float64
dtypes

В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [3]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.shape

Unique values of ClaimInd: [0]


(967, 2)

In [4]:
NegClaimAmount.head()

,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [5]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [6]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [7]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [8]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

**NB**: В H2O не рекомендуется использовать one-hot encoding, поскольку данный фреймворк корректно работает с категориальными признаками, тогда как применение one-hot encoding приводит к неэффективности. Тем не менее, используем здесь фиктивные переменные, чтобы в дальнейшем сохранить возможность сравнения результатов построенных моделей.

In [9]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [10]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [11]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43',
       'CSP39', 'CSP5', 'CSP32', 'CSP35', 'CSP73', 'CSP62', 'CSP52',
       'CSP27', 'CSP24', 'CSP19', 'CSP70'], dtype=object)

In [12]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [13]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [14]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [15]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [16]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,0,1,0,3249


Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [17]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [18]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [20]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from C:\Users\a.andoskin\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\A7343~1.AND\AppData\Local\Temp\tmpto99zgxa
  JVM stdout: C:\Users\A7343~1.AND\AppData\Local\Temp\tmpto99zgxa\h2o_a_andoskin_started_from_python.out
  JVM stderr: C:\Users\A7343~1.AND\AppData\Local\Temp\tmpto99zgxa\h2o_a_andoskin_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_a_andoskin_12cmi4
H2O cluster total nodes:,1
H2O cluster free memory:,1.770 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [21]:
h2o_df = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,OutUseNb,...,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq,ClaimsCount
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,0,0,0,0,0,1,0,0,2116,0.0
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,0,0,0,0,0,1,0,0,2116,0.0
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,1,0,0,1024,0.0
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,1,0,0,2704,0.0
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,3249,0.0


In [24]:
df['ClaimInd'].value_counts()

0    104286
1     10869
Name: ClaimInd, dtype: int64

In [25]:
(10869 * 100) / (104286 + 10869)

9.438582779731666

Целевая переменная не сбалансирована.

In [43]:
# Разбиение датасета на train/val/test

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_train_ind, y_train_ind, test_size=0.5, random_state=1)

In [44]:
# Преобразование в H2O-Frame

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [45]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

h2o_train_ind['ClaimInd'] = h2o_train_ind['ClaimInd'].asfactor()
h2o_valid_ind['ClaimInd'] = h2o_valid_ind['ClaimInd'].asfactor()
h2o_test_ind['ClaimInd'] = h2o_test_ind['ClaimInd'].asfactor()

In [46]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_binomial = H2OGeneralizedLinearEstimator(family = "binomial", nfolds=5)
glm_binomial.train(y="ClaimInd", x = h2o_train_ind.names[1:-1], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [47]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_binomial.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 2.368E-5 )",20,20,3,py_4_sid_9931


In [48]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_binomial.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.547919,0.08244557,0.61696786,0.42785516,0.53604215,0.5251096,0.63362014
1,auc,0.56286985,0.006894399,0.5654949,0.5543302,0.5587669,0.5725319,0.56322545
2,aucpr,0.113643676,0.0018182492,0.11432205,0.112301566,0.11575254,0.11457506,0.11126715
3,err,0.45208102,0.08244557,0.38303217,0.57214487,0.46395782,0.4748904,0.36637986
4,err_count,7287.0,1323.9227,6253.0,9243.0,7479.0,7584.0,5876.0
5,f0point5,0.13463415,0.003396567,0.13739729,0.130117,0.13834237,0.132551,0.13476314
6,f1,0.18718885,0.0036467107,0.18697178,0.18671359,0.19276848,0.18696398,0.18252644
7,f2,0.30812988,0.019759703,0.29251423,0.33044723,0.3177936,0.3171601,0.28273425
8,lift_top_group,1.4666644,0.27526635,1.5583981,1.0208288,1.4348853,1.5585115,1.7606986
9,logloss,0.3112646,0.007344858,0.3090959,0.3164358,0.32124746,0.3052057,0.30433822


In [49]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_binomial._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.534238,-2.279643
1,LicAge,-0.000312,-0.049906
2,Gender,0.010311,0.004998
3,MariStat,-0.066143,-0.023827
4,DrivAge,-0.001085,-0.016265
5,HasKmLimit,-0.366059,-0.114316
6,BonusMalus,0.006341,0.097335
7,OutUseNb,0.061289,0.042639
8,RiskArea,0.009015,0.019975
9,VehUsage_Private,-0.146273,-0.069170


In [50]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_binomial.coef_norm()
for x in range(len(glm_binomial.cross_validation_models())):
    pmodels[x] = glm_binomial.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
BonusMalus,0.09734,0.09532,0.10271,0.09799,0.09584,0.09390
DrivAge,-0.01626,0.00000,-0.00825,-0.06719,-0.03556,-0.00207
DrivAgeSq,0.01208,0.00000,0.00000,0.03765,0.03064,0.02684
Gender,0.00500,0.00913,0.00000,0.00593,-0.00676,0.01718
HasKmLimit,-0.11432,-0.10352,-0.11824,-0.12745,-0.11287,-0.11097
Intercept,-2.27964,-2.27682,-2.28745,-2.29369,-2.27133,-2.27078
LicAge,-0.04991,-0.05077,-0.04192,-0.01125,-0.06192,-0.08503
MariStat,-0.02383,-0.01981,-0.01768,-0.01872,-0.03119,-0.03285
OutUseNb,0.04264,0.04081,0.04933,0.04794,0.03546,0.03986
RiskArea,0.01997,0.01695,0.02583,0.02213,0.01517,0.02031


In [51]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_binomial.predict(h2o_train_ind).as_data_frame()
ind_valid_pred = glm_binomial.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_binomial.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [52]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [53]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

accuracy_score(y_train_ind, ind_train_pred['predict']), \
accuracy_score(y_valid_ind, ind_valid_pred['predict']), \
accuracy_score(y_test_ind, ind_test_pred['predict'])

(0.4550913060738388, 0.45608376339817386, 0.4540988487495038)

In [54]:
f1_score(y_train_ind, ind_train_pred['predict']), \
f1_score(y_valid_ind, ind_valid_pred['predict']), \
f1_score(y_test_ind, ind_test_pred['predict'])

(0.18614044839725774, 0.18915520047344284, 0.18311427934952104)

In [55]:
print(f"{confusion_matrix(y_train_ind, ind_train_pred['predict'])} \n"  \
f"{confusion_matrix(y_valid_ind, ind_valid_pred['predict'])} \n" \
f"{confusion_matrix(y_test_ind, ind_test_pred['predict'])}")

[[31661 41312]
 [ 2612  5023]] 
[[15825 20629]
 [ 1293  2557]] 
[[15836 20683]
 [ 1319  2466]]


Какие проблемы вы здесь видите? Как можно улучшить данный результат?

1. train_test_split ввести параметр "stratify"? чтобы разбиение целевой переменной было прапорционально
2. Т.к. у нас не сбалансированная выборка, "1" составляет около 10% можно ввести балансировку классов  
3. Произвести предварительную очистку в данных  
4. Использовать другое распределение
5. Использование нелинейных моделей

Ввведем в train_test_split параметр "stratify"

In [57]:
x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimInd, test_size=0.3, random_state=1, stratify=df.ClaimInd)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_train_ind, y_train_ind, test_size=0.5, random_state=1, stratify=y_train_ind)

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

glm_binomial = H2OGeneralizedLinearEstimator(family = "binomial", nfolds=5)
glm_binomial.train(y="ClaimInd", x = h2o_train_ind.names[1:-1], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind)

glm_binomial.cross_validation_metrics_summary().as_data_frame()

ind_train_pred = glm_binomial.predict(h2o_train_ind).as_data_frame()
ind_valid_pred = glm_binomial.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_binomial.predict(h2o_test_ind).as_data_frame()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [58]:
accuracy_score(y_train_ind, ind_train_pred['predict']), \
accuracy_score(y_valid_ind, ind_valid_pred['predict']), \
accuracy_score(y_test_ind, ind_test_pred['predict'])

(0.47692536720921, 0.47846367606192935, 0.47538705835649064)

In [59]:
f1_score(y_train_ind, ind_train_pred['predict']), \
f1_score(y_valid_ind, ind_valid_pred['predict']), \
f1_score(y_test_ind, ind_test_pred['predict'])

(0.1862116884119509, 0.18822893334363172, 0.18419631144378423)

In [60]:
print(f"{confusion_matrix(y_train_ind, ind_train_pred['predict'])} \n"  \
f"{confusion_matrix(y_valid_ind, ind_valid_pred['predict'])} \n" \
f"{confusion_matrix(y_test_ind, ind_test_pred['predict'])}")

[[33620 39380]
 [ 2784  4824]] 
[[16847 19653]
 [ 1367  2437]] 
[[16773 19727]
 [ 1417  2387]]


В результате введения в train_test_split параметра "stratify"  
Получили прирост accuracy, незначительный прирост f1 score, и наблюдаем что ошибок первого и второго рода в confusion matrix стало меньше